In [3]:
import sys
# Define relative path.
sys.path.append("python")

In [4]:
import numpy as np

from scipy.stats import norm
from numpy.testing import assert_array_equal
from numpy.testing import assert_array_almost_equal

from transform_distributions import transform_uniform_stnormal_uncorr
from transform_distributions import covariance_to_correlation
from transform_distributions import transform_stnormal_normal_corr_lemaire09

sys.path.append("python/tests/resources/test_transform_distributions")
from nataf_transformation import nataf_transformation
from distributions import distributions

In [3]:
# Define shared objects for the next two tests.
# Expectation values.
mu = np.array([10, 10, 10, 10, 10])

# Covariance matrix.
cov = np.array([
        [10,0,0,2,0.5],
        [0,20,0.4,0.15,0],
        [0,0.4,30,0.05,0],
        [2,0.15,0.05,40,0],
        [0.5,0,0,0,50]])

# Draws from U(0,1).
row = np.array([0.1, 0.1, 0.2, 0.8, 0.5])
# Transform draws to uncorrelated N(0,1).
z = transform_uniform_stnormal_uncorr(row)

In [4]:
# Create Nataf transformation from class for many distribution types.
M = list()
M.append(distributions('normal', 'PAR', [mu[0], np.sqrt(cov[0 ,0])]))
M.append(distributions('normal', 'PAR', [mu[1], np.sqrt(cov[1 ,1])]))
M.append(distributions('normal', 'PAR', [mu[2], np.sqrt(cov[2 ,2])]))
M.append(distributions('normal', 'PAR', [mu[3], np.sqrt(cov[3 ,3])]))
M.append(distributions('normal', 'PAR', [mu[4], np.sqrt(cov[4 ,4])]))

Rho = covariance_to_correlation(cov)

T_Nataf = nataf_transformation(M, Rho)

In [11]:
x_lemaire09, _ = transform_stnormal_normal_corr_lemaire09(z, cov, mu)
X = T_Nataf.U2X(z).T

In [12]:
X

array([[ 5.94737811,  4.26872717,  5.27623987, 14.4353868 ,  9.78399446]])

In [13]:
x_lemaire09

array([[ 5.94737811,  4.26872717,  5.27623987, 14.4353868 ,  9.78399446]])

In [8]:
# Reverse order in mu and cov.
row_rev = row.T

mu_rev = mu.T

cov_rev = np.array([
    [50, 0, 0, 0, 0.5],
    [0, 40, 0.05, 0.15, 2],
    [0, 0.05, 30, 0.4, 0],
    [0, 0.15, 0.4, 20, 0],
    [0.5, 2, 0, 0, 10]])

# Transform draws to uncorrelated N(0,1).
z_rev = transform_uniform_stnormal_uncorr(row_rev)

In [14]:
x_lemaire09_rev, _ = transform_stnormal_normal_corr_lemaire09(z_rev, cov_rev, mu_rev)

# The order does matter.

In [15]:
x_lemaire09_rev

array([[ 0.93806198,  1.89475623,  5.3801239 , 13.67146099,  9.5030972 ]])